In [1]:
import pandas as pd
import numpy as np
import CleanData as cd
import BokehPlots as bp
import CoinagePlots as cp
import sqlite3

# import sys
# stdout = sys.stdout
# reload(sys)
# sys.setdefaultencoding('utf-8')
# sys.stdout = stdout

## TODO:
* Write Documentation for CoinagePlots
* 'Radiate crown' in Augustian Era

In [2]:
title = 'AugustusCoins_44BC-14AD'
fname = title+'.sqlite'

# Read in Data

### British Museum

In [3]:
museum = 'britishMuseum'
bad_locations = ['Alexandria', 'Cos', 'Apamea', 'Philippi', 'Thrace', 
                 'Achulla', 'Arabia', 'Orthosia', 'Sebaste', 'Prymnessus', 
                 'Acmonea', 'Antiochia', 'Eucarpea', 'Chalkis', 'Apollonia', 
                 'Midaeum', 'Magnetes', 'Heraclea Salbace', 'Gaul', 'Bilbilis|Italica',
                 'Spain II', 'Lyon', '']
conds = [['NOT IN', 'mint', bad_locations]]
mints_bm = cd.readQuery(fname, museum, conditions=conds)
mints_bm['dates'] = mints_bm['dates'].apply(cd.stringToList(valType=int))
mints_bm.head()

,associatedNames,authority,bibliography,culture,curatorComment,dates,denomination,description,materials,museumNumber,objectType,obverseLegend,mint,reverseLegend,state,subjects,weight,url,source
0,Augustus (Octavian),Augustus (Octavian),RPC1 1801,,,"[-27, 14]",?,Bronze coin.(obverse) Head of Augustus r. (rev...,bronze,"1997,1206.1",coin,,Odessus,ODE,Roman Empire,"symbol, emperor/empress",7.45,http://www.britishmuseum.org/research/collecti...,British Museum
1,,Augustus (Octavian),,Greek,,"[-27, 14]",?,Alloy coin.,alloy,"1905,1111.1",coin,,Cremna,,,,1.49,http://www.britishmuseum.org/research/collecti...,British Museum
2,,Augustus (Octavian),,Greek,,"[-27, 14]",?,Alloy coin.,alloy,"1936,0220.13",coin,,Tyndaris,,,,7.41,http://www.britishmuseum.org/research/collecti...,British Museum
3,,Augustus (Octavian),,Roman Provincial,,"[-27, 14]",?,Alloy coin.,alloy,G.329,coin,,Africa,,Roman Empire,,29.02,http://www.britishmuseum.org/research/collecti...,British Museum
4,,Augustus (Octavian),,Roman Provincial,,"[-27, 14]",?,Alloy coin.,alloy,G.328,coin,,Mauretania,,Roman Empire,,27.54,http://www.britishmuseum.org/research/collecti...,British Museum


In [ ]:
museum = 'americanNumismaticSociety'
bad_locations = ['', 'uncertain', 'Uncertain value', 'Bilbilis|Italica']
conds = [['NOT IN', 'mint', bad_locations]]
mints_ans = cd.readQuery(fname, museum, conditions=conds)

# Plot Data
* Plot all data together in plots
* Split up data and plot seperately

### British Museum

In [5]:
from bokeh.io import output_notebook, save
from bokeh.plotting import show

In [ ]:
#output_notebook()

In [ ]:
location_bar_plot = cp.makeCoinageStackedBar(mints_bm, title='Number of coins produced from each location')
# save(location_bar_plot, filename='../Plots/location_bar.html')
show(location_bar_plot)

In [ ]:
location_map_plot = cp.makeCoinageMap(mints_bm, title='Number of coins produced from each location', 
                                   pt_size=lambda x: 5 * np.log(3 * x), colors_ascending=False)
#save(location_map_plot, filename='../Plots/location_map.html')
show(location_map_plot)

### Individual time periods and subjects

In [6]:
from bokeh.layouts import gridplot

In [16]:
mints_with_desc = mints_bm[mints_bm['description'] != '']
date_ranges = [(-44, -31), (-30, -27), (-27, -19), (-18, -16), (-15, -11), (-10, 13)]
subjects = [(['star'], ['description']), (['statue of Augustus'], ['description']), 
            (['capricorn', 'globe', 'rudder'], ['description' for _ in range(3)]), 
            (['Julius Caesar'], ['associatedNames']), 
            (['Apollo', 'lyre'], ['description', 'description'])]
good_dfs = {}
grid_plots = []

for dates in date_ranges:
    for subject in subjects:
        plot_title = cp.makeTitle(dates, subject[0])
        contain_dates = cp.coinsFromDates(mints_with_desc, dates, col_name='dates')
        good_df = cp.containKeyword(contain_dates, subject[0], subject[1])
        
        if not good_df.empty:
            good_dfs[plot_title] = good_df
            bar_plot = cp.makeCoinageStackedBar(good_df, 'mint', 'denomination', colors=viridis, 
                                                title=plot_title+' Stacked Bar Graph', plot_size=(480, 480))
            map_plot = cp.makeCoinageMap(good_df, 'mint', pt_size=lambda x: 10 * x, title=plot_title+' Map')
            grid_plots.append([bar_plot, map_plot])

grid = gridplot(grid_plots)
# Saving not supported so show plot and save the page
# show(grid)
print(mints_with_desc)

                                       associatedNames  \
0                                  Augustus (Octavian)   
1                                                        
2                                                        
3                                                        
4                                                        
5                                                        
6                                                        
7                                                        
8                                                        
9                                                        
10                                 Augustus (Octavian)   
11                       Tiberius, Augustus (Octavian)   
12                                                       
13                                                       
14                                                       
15                                                       
16            

In [20]:
subjects = [(['radiate'], ['description'])]
grid_plots = []
good_dfs = {}

for subject in subjects:
    plot_title = cp.makeTitle([-44, 13], subject[0])
    good_df = cp.containKeyword(mints_with_desc, subject[0], subject[1])

    if not good_df.empty:
        good_dfs[plot_title] = good_df

        bar_plot = bp.makeStackedBar(good_df, 'Production place', 'Denomination', sort_bars=True,
                               bars_ascending=False, sort_stacks=True, stacks_agg='sum', stacks_ascending=False,
                              colors=viridis, title=plot_title+' Stacked Bar Graph', plot_size=(480, 480))
        bar_plot.yaxis.axis_label='Location Counts'
        bar_plot.add_tools(HoverTool(tooltips=[('Denomination', '@Denomination'), 
                                                        ('Denomination Count', '@height'),
                                                        ('Location Count', '@Sum')]))

        map_counts = cd.prepareDataframeForMapping(good_df)
        map_plot = bp.makeMap(map_counts, 'Production place', 'Count', x_ranges=(-2.0e6, 5e6), 
                            y_ranges=(3.5e6, 7e6), mintsFile='../GeoJSON/mints.geojson', path='../GeoJSON/', ext='html', 
                             title=plot_title+' Map', colors_ascending=False, pt_size=lambda x: 5 * np.log(3 * x))
        grid_plots.append([bar_plot, map_plot])

grid = gridplot(grid_plots)
#save(grid, filename='../Plots/Many plots')
show(grid)